# Joining all datasets with generalized column names

- With hate not included
- What is thumbs up supposed to mean in by_fans page
- Random is included but party will be random
- 0:Unbiased, 1: far-left, 2:centre-left, 3:centre, 4: centre-right, 5:far-right
- Type signifies each page in polly dataset
    - Type 5 is hate and is not included because it does not have party information


In [125]:
import pandas as pd
import glob
import re

In [126]:
def preprocess(row):
    result = re.sub(r"http\S+", "", row['Tweet'])
    result = re.sub(r"RT @\S+", "", result)
    result = re.sub(r"@\S+", "", result)
    result = re.sub(r"\n", "", result)
    return result

In [127]:
data = pd.DataFrame()

In [128]:
page = pd.read_csv('../data/polly/about_party.csv')
page = page.rename(columns={'♡': 'Likes'})
page = page.rename(columns={'About': 'Party'})
page['Type'] = 1
page = page[['Tweet', 'Likes', 'Party', 'Type']]
data = data.append(page, ignore_index=True)

In [129]:
page = pd.read_csv('../data/polly/by_party.csv')
page = page.rename(columns={'♡': 'Likes'})
page = page.rename(columns={'By': 'Party'})
page['Type'] = 2
page = page[['Tweet', 'Likes', 'Party', 'Type']]
data = data.append(page, ignore_index=True)

In [130]:
page = pd.read_csv('../data/polly/by_fans.csv')
page = page.rename(columns={'♡': 'Likes'})
page = page.rename(columns={'👍': 'Party'})
page['Type'] = 3
page = page[['Tweet', 'Likes', 'Party', 'Type']]
data = data.append(page, ignore_index=True)

In [131]:
page = pd.read_csv('../data/polly/with_emoji.csv')
page = page.rename(columns={'♡': 'Likes'})
page = page.rename(columns={'About': 'Party'})
page['Type'] = 4
page = page[['Tweet', 'Likes', 'Party', 'Type']]
data = data.append(page, ignore_index=True)

In [132]:
page = pd.read_csv('../data/polly/with_isa.csv')
page = page.rename(columns={'♡': 'Likes'})
page = page.rename(columns={'About': 'Party'})
page['Type'] = 6
page = page[['Tweet', 'Likes', 'Party', 'Type']]
data = data.append(page, ignore_index=True)

In [133]:
page = pd.read_csv('../data/polly/random.csv')
page = page.rename(columns={'♡': 'Likes'})
# page = page.rename(columns={'👍': 'Party'})
page['Type'] = 7
page['Party'] = 'random'
page = page[['Tweet', 'Likes', 'Party', 'Type']]
data = data.append(page, ignore_index=True)

In [134]:
data['text'] = data.apply(preprocess,axis=1)

In [135]:
data.shape

(106048, 5)

In [136]:
data = data.dropna()

In [137]:
data = data[data['Party'].notnull()]
data = data[data['text'].notnull()]

## Reducing data to just to include two extremes

In [138]:
data = data[data['Party'].isin(['AfD','Die Linke'])]
data = data[data['Party'].notnull()]
data['labels'] = 0
data.loc[data.Party == 'Die Linke', ['labels']] = 0
data.loc[data.Party == 'AfD', ['labels']] = 1
# data = data.rename(columns={'Label':'labels'})

In [139]:
data.to_csv('../data/polly/polly.csv')

In [121]:
data = pd.read_csv('../data/polly/polly.csv')
del data['Unnamed: 0']

In [140]:
data_0 = data[data['Party']=='Die Linke']
data_1 = data[data['Party']=='AfD']
data_1 = data_1.sample(data_0.shape[0])

In [141]:
print(data_0.shape)
print(data_1.shape)

(8185, 6)
(8185, 6)


In [142]:
train= pd.DataFrame()
train = train.append(data_0.sample(frac=0.9))
test = pd.DataFrame()
test= test.append(data_0.drop(train.index))
train = train.append(data_1.sample(frac=0.9))
test= test.append(data_1.drop(train[train['Party']=='AfD'].index))

train.to_csv('../data/polly/polly_train1.csv')
test.to_csv('../data/polly/polly_test1.csv')

In [143]:
print(train[train['Party'] == 'AfD'].shape)
print(train[train['Party'] == 'Die Linke'].shape)

(7366, 6)
(7366, 6)


In [144]:
test.shape

(1638, 6)

In [99]:
train = pd.read_csv('../data/polly/polly_train.csv')
train.shape


(17031, 7)

## Test results on binary data

### For model v2 for unbalanced classes

In [185]:
predicted = pd.read_csv('../data/polly/polly_text_predicted_gelectra_v1.csv')

In [186]:
s = predicted[predicted['labels'] == predicted['prediction']]

In [187]:
(s.shape)[0]/(predicted.shape)[0]

0.6235321747299202

In [ ]:
# train with preprocessing
# learns party specific may not be based on the semantics
# Try zero shot with available models
# Use the model with other parties to see threshold based agreement

## Other parties data

In [17]:
data = data[data['Party'].notnull()]
data = data[data['text'].notnull()]

In [ ]:
del data['labels']

# predicition on different parties

In [18]:
data = data[data['Party'].isin(['FDP','CDU','Die Grünen','SPD','CSU','random'])]
data = data[data['Party'].notnull()]
data['label'] = 0
data['text'] = data.apply(preprocess,axis=1)

In [19]:
data['Party'].unique()

array(['FDP', 'CDU', 'Die Grünen', 'SPD', 'CSU', 'random'], dtype=object)

In [86]:
data.to_csv('../data/polly/other_parties.csv')

In [21]:
predicted = pd.read_csv('../data/polly/other_parties_predicted.csv')

In [22]:
import numpy as np
p = predicted[predicted['Party']=='random']
p_right = p[p['prediction']==1]
print('Percentage right : ', p_right.shape[0]/ p.shape[0])
p_left = p[p['prediction']==0]
print('mean probability for right: ',np.mean(np.array(p_right['probabilities'].tolist())))
print('std: ',np.std(np.array(p_right['probabilities'].tolist())))
print('median: ', np.median(np.array(p_right['probabilities'].tolist())))
print('Mean probability for left: ',np.mean(np.array(p_left['probabilities'].tolist())))

Percentage right :  0.46818067332717656
mean probability for right:  0.9557483476779284
std:  0.09982604482581645
median:  0.9978795051574708
Mean probability for left:  0.03745034753988003


FDP :
Percentage right :  0.5189061886693707
mean probability for right:  0.9631497984544473
std:  0.08861969671054513
median:  0.998441219329834
CDU :
Percentage right :  0.5558645347104976
mean probability for right:  0.970054559927445
std:  0.08430349321980597
median:  0.999220848083496
Mean probability for left: 0.03478617466948551
Die Grünen : 
Percentage right :  0.5072124756335282
mean probability for right:  0.9637299863888367
std:  0.09324209251693516
median:  0.9986621141433716
Mean probability for left:  0.03821723985493388
SPD : 
Percentage right :  0.5511957278848386
mean probability for right:  0.9657167397947902
std:  0.08788509247678603
median:  0.998898983001709
Mean probability for left:  0.03519495763810857
CSU :
Percentage right :  0.6351125938281902
mean probability for right:  0.9697873348165952
std:  0.08227020757262209
median:  0.9990532994270324
Mean probability for left:  0.04239499288147112
random :
Percentage right :  0.46818067332717656
mean probability for right:  0.9557483476779284
std:  0.09982604482581645
median:  0.9978795051574708
Mean probability for left:  0.03745034753988003

In [89]:
p.head()

,Unnamed: 0,Unnamed: 0.1,Tweet,Likes,Party,Type,text,label,prediction,probabilities
0,0,0,"""Die FDP ist noch nicht über den Berg und sollte Demut bewahren"", mahnt der Parteigrande Hermann-Otto Solms http://",0,FDP,1,"""Die FDP ist noch nicht über den Berg und sollte Demut bewahren"", mahnt der Parteigrande Hermann-Otto Solms",0,1,0.991131
1,1,1,Solms warnt FDP nach Dreikönigstreffen vor zu viel Hochmut: Der FDP-Bundesschatzmeister Hermann-Otto Solms (FDP)… http:// http://,0,FDP,1,Solms warnt FDP nach Dreikönigstreffen vor zu viel Hochmut: Der FDP-Bundesschatzmeister Hermann-Otto Solms (FDP)…,0,1,0.999892
2,2,2,Da hat Marco Buschmann vollkommen recht... cv http://,0,FDP,1,Da hat Marco Buschmann vollkommen recht... cv,0,0,0.000933
3,3,3,Live vom Neujahrsempfang der FDP mit Hermann Otto Solms http://,0,FDP,1,Live vom Neujahrsempfang der FDP mit Hermann Otto Solms,0,1,0.968151
4,4,4,"Leser 2017: Marco Buschmann, Bundesgeschäftsführer der FDP http://",0,FDP,1,"Leser 2017: Marco Buschmann, Bundesgeschäftsführer der FDP",0,0,0.000313


# Analysing the failures using v2 so as to identify patterns

In [145]:
test_predicted = pd.read_csv('../data/polly/polly_predicted.csv')

In [146]:
test_predicted.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Tweet', 'Likes', 'Party', 'Type',
       'labels', 'text', 'prediction', 'probabilities'],
      dtype='object')

In [167]:
wrong_predictions = pd.DataFrame()
intermediate = test_predicted[test_predicted['Party']=='AfD']
intermediate = intermediate[intermediate['prediction']==0]
print(intermediate.shape)
wrong_predictions = wrong_predictions.append(intermediate, ignore_index=True)
intermediate = test_predicted[test_predicted['Party']=='Die Linke']
intermediate = intermediate[intermediate['prediction']==1]
print(intermediate.shape)
wrong_predictions = wrong_predictions.append(intermediate, ignore_index=True)

(329, 10)
(359, 10)


In [190]:
from deep_translator import GoogleTranslator
def translate_to_english(row):
    if (not len(row['text'])>5000):
        if(len(row['text'].strip())>0):
            print('---------------------',row['text'])
            english_text = GoogleTranslator(source='de', target='en').translate(row['text'])
    #         print(english_text)
            return english_text
    else:
        return None
wrong_predictions['english_text'] = wrong_predictions.apply(translate_to_english, axis=1)

---------------------   "Solidaritätsdemonstration" nennt der normale Mensch: Unterstützung von Linksterrorismus! #noRAF
---------------------   In einem Salat sind verschiedene Zutaten,warum nicht auch Schnittlauch ?
---------------------  Ich kenne da noch eine Grüne, die hat an einer noch ganz anderen  Demonstration teilgenommen. Ist wohl da ne übliche  Freizeitbeschäftigung.
---------------------   Versuch mal bitte einen Vorteil aufzuzeigen den eine AfD-Wahl hätte.
---------------------    Ich glaube da könnt Ihr lange drauf warten!
---------------------  #ehrlicher #Feind,Lieber als #Falscher #Freund ,ihr wisst was ich mein dann kennt ihr die Farbe vom 🐗und der 🐽#haram außer Himmelblau #BTW17 
---------------------   Schiebt euer illegaler Parteispendeverein, Goal AG, eigentlich Überstunden, um die vielen Grafiken zu produzieren?
---------------------  Grenzen müssen man schützen - ist klar.Sie sollten nüchtern twittern und den Alkohol weglassen.
---------------------   Leider gi

--------------------- Gefängnisstrafe gegen "verbale Angriffe" weil "Argumente" nicht mehr helfen. Ihr habt keine Argumente!! #Nordkorea 
--------------------- Ich wollte doch nur arbeiten! 
--------------------- Gleich geht's los in #Gütersloh!Viele Grüße
--------------------- Start in Rodgau mit  vielen Interessierten im Saal und ein paar Schreihälsen draußen.Wahlkampf macht Spaß.Bundestag-wir kommen! 
--------------------- Ärtzte-ohne-Grenzen-Chef gibt zu: wir retten Menschen, die sich selbst absichtlich in Gefahr bringen. Und die Zukunft ist ihm egal.  
--------------------- Massenmord durch unseren Handelspartner und geschätzten Waffenkäufer Saudi-Arabien. #Aufschrei ? 
---------------------  Solche Phrasen dürfen keine Entschuldigung für die Regierung sein, sich nicht anzustrengen!
--------------------- Überraschung 😀 
---------------------  Beschäftige mich als ehem. Oberst viel mit Sicherheit und Außenpolitik. Aber danke für Ihre Anregung, soziale Themen sind sehr wichtig!
----

--------------------- Wann werden Vorstandsmitglieder von Firmen wie MONSANTO und Bayer vor Gericht gestellt? 
---------------------   1/3 Europa ist ein großer Kultur-Raum, der sich aus vielen kleineren Kultur-Räumen zusammensetzt, die sich aber ähneln.
---------------------  Gibt es einen Link auf diesen Kommentar ? Sieht sehr nach Photoshop bearbeitet aus...
---------------------    Meinungen und Ansichten ändern sich indem man sich damit so intensiv befasst daß sich die Nervenbahnen anders einschleifen. Aber es ist
---------------------    Vielleicht einfach mal sämtliche Zuschüsse streichen. Ärzte innerhalb von Grenzen finde ich zudem weitaus sinnvoller.
---------------------   Das glaubst auch nur du. Ich wohnte neben einer Anlage von 30 Windkaftanlagen.Im Herbst zum Vogelzug Schwäne, Milane und sogar Störche !
---------------------   also in 8-12 Jahren?
--------------------- Fundstück: Wie baut man einen #Staat in eine linke #Diktatur um? Parallelen zu bestehenden Ländern... 
-

--------------------- BILD nimmt Umfrage raus. AFd zu stark? SPD, FDP, CDU, die Linke zu tief?  via 
--------------------- Für gute Arbeit, gutes Geld! In diesem Fall jedoch nicht! 🤔🆘🇩🇪 
--------------------- Norbert Bolz im Gespräch mit Wolfgang Herles  via 
--------------------- Von Müll kommt Müll! 
---------------------   Kein Mensch braucht die Linke....
---------------------    Das Hätschel-Kind der LINKEN
--------------------- Die alten mielke Seilschaften der im-erika? 
---------------------   Jetzt 20 %, bald 33 %
--------------------- Ich will nur noch ausrasten.Jeden Tag... 
--------------------- Unglaublich 
---------------------   Die vergessen dabei immer wieder, dass der Wähler anders tickt. Hier wird es Verlierer geben. Für die Grünen und die FDP wäre es endgültig.
---------------------   Sehr wichtig und richtig
--------------------- Ich will das nicht!
--------------------- Potentieller SeuchenherdBerlin z.B bei HIV/AIDS, Hepatitis B, Legionärskrankheit, Meningokokken

--------------------- so viele wörter sind verbraucht, also sag ich weiter yo, denn es fördert den verkauf.
--------------------- Gehalt für Abgeordnete: Diäten sollen weiter automatisch steigen.  
--------------------- Krisensitzung im Rathaus von Falkenstein (Vogtlandkreis) - Kleinstadt streitet über ihre Nazi-Turnhalle    Wer ständig den Dreck von hinten nach vorne schaufelt,stolpert irgendwann darüber.
---------------------  Sehr geehrter Italo, wie heißt die Quelle (idealerweise in Deutsch)? Ich fand das Schreiben von Hilaire Belloc immer sehr interessant. Danke. Gruß
--------------------- Unsere letzten Regierungen waren einfach  zu blöde zu regieren und liebers wird das Geld der Solidargemeinschaft unnütz ausgegeben , für Menschen die nicht in Deutschland leben.       
---------------------   Gerne. Danke auch. Fürs Mitmachen. Auf weiter guten Zusammenhalt! Herzliche Grüße an Georg, Marc & Kollegen! 👍
--------------------- Vielen herzlichen Dank an all die Mitstreiter, die mir h

--------------------- „Wir brauchen in Deutschland einen sozialen Schub“, sagt  
--------------------- Meine Güte: SPD hat der Union in den letzten Jahren so wenig Contra gegeben. Kritik an #Nahles wird langsam albern.
--------------------- Hier sieht man die Seele der AfD bei der Arbeit, wie es Herr Gauland ungefähr formulierte. Gebürtig und Sozialisiert in Westdeutschland! 
--------------------- Hier noch für meine treuen Follower der Bayernpartei. Die Übernahme Restdeutschlands hat nicht geklappt. PS: fahre gerade nach München
--------------------- Großartig!  
--------------------- Ein furchtbarer Mord, der aufgeklärt werden muss! Und warum haben BMW, BASF & Lufthansa Briefkastenfirmen in #Malta? 
--------------------- #FabioDeMasi im SWR: Besser für #Europa, wenn nicht nur Entscheidungen getroffen werden, die nur #Merkel schmecken 👍  
---------------------  Sie kennen offensichtlich das Jahr 1989 in Deutschland nicht und die Rolle der Bürger in der DDR die friedlich sich ihren Weg

---------------------   Wir sind alle Ochsen vor Gott (Brian Molko)
---------------------   Zumindest, seit Süditalien in die Schulpflicht genommen wurde.
---------------------  Du scheinst den Film aufs Korn zu nehmen ^^
---------------------  Mechaniker... weiß mehr als tausend Wissenschaftler. Genau
---------------------  P.S. Ich hatte auf seinen Tweet geantwortet. Nicht, dass ihr denkt ich wär besonders toll oder  folgt mir. SO cool bin ich nicht😆
--------------------- Auch in der Mediathek... ;-) 
--------------------- Schleschlaaz  Schlechtester Schlag aller Zeiten  #schlefaz
---------------------   Ja modernes Haus und lahmes Internet auf dem Land. Dazu noch dieser schwachsinnige hashtag #fedidwgugl
---------------------  Geb ich gleich weiter. Der funkt noch fleissig. 😃
--------------------- Das selbe Lied Provozieren, dementieren. #noracism #noAfD #Gauland mit Hass gefüllter Rassist ohne Antworten auf Herausforderung - nur Hetze
---------------------   #Kartoffelsuppe nicht v

--------------------- Wenn #AfD'ler über #Flüchtlingspolitik sprechen, isr das immer ein einziges Trauerspiel... 😒☹ #wirwaehlen
---------------------  Oh ja, das sind die #InterRegio... ☹🙄 Fürchterlich... Marode Bahn..
--------------------- Das ist nur noch ein einziges Trauerspiel, was hier mit #Griechenland passiert… ☹
---------------------   Mittendrin im Normalisierungsdiskurs wird dann gefragt, ob man nicht ihre Sorgen um den Volkskörper ernst nehmen sollte, Kompromisse suchen.
---------------------   Wolffsohn hat Kluges zum dt. Antisemitismus gesagt & vertritt dennoch als BW-Uni-Dozent ein geschöntes Staatsräson-Bild vom geläuterten Dtl.
---------------------  Das Verhalten von SA & SS vor der Machtübernahme lässt sich gut mit Pegida, autonomen Kameradschaften und anderen AfD-Konsorten vergleichen.
---------------------   weiß mehr ...
---------------------  Ich wollte ja auch nicht alle wegjagen ;-) Da bin ich dann doch Opportunist genug.
---------------------  Mir fällt es lei

--------------------- Die #Wirtschaftsweisen wollen den #Feierabend abschaffen. Nicht mit uns! Die Antwort auf die… 
--------------------- Eine Krähe hackt einer anderen kein Auge aus. 
--------------------- Es ist an der #Zeit, daß wir uns daran erinnern,daß es auch in #Deutschland einmal eine #Friedensbewegung gegeben hat!Es ist an der Zeit, daß wir den Amerikanern ihre #Atomwaffenmit Dank zurückgeben.Gebt dem #Kaiser was des Kaisers ist!
---------------------   Anschein und Fakten sind halt zweierlei
--------------------- Es gibt jetzt eine Barbie mit Hidschab 
--------------------- Und was taten u. tun unsere Volksvertreter, die diese Bezeichnung mit Sicherheit nicht verdienen? Nicht viel, weil  mit dem Problem müssen sich spätere Regierungen auseinandersetzen.. 
--------------------- Was ein Quatsch!Skandal in französischer TV-Show: Lagerfeld nennt Flüchtlinge "schlimmste Feinde" der Juden  via 
---------------------    Das muss man nicht, ganz im Gegenteil!Wer friedlichen Demonst

---------------------  👍✊!?Wie die Folgen offener Grenzen finanziert werden sollen,dazu sagt das Programm nichts.
--------------------- Bis 59 Jahre wär  locker drin. Darüber kippt es unter die 5%. (Mehr Ost wär gut gewesen ;) Bei unter 25jährigen 9%. 👍 


In [191]:
wrong_predictions.to_csv('../results/wrong_predictions_v2_to_analyse.csv')

In [192]:
correct_predictions = pd.DataFrame()
intermediate = test_predicted[test_predicted['Party']=='AfD']
intermediate = intermediate[intermediate['prediction']==1]
print(intermediate.shape)
correct_predictions = correct_predictions.append(intermediate, ignore_index=True)
intermediate = test_predicted[test_predicted['Party']=='Die Linke']
intermediate = intermediate[intermediate['prediction']==0]
print(intermediate.shape)
correct_predictions = correct_predictions.append(intermediate, ignore_index=True)

(2326, 10)
(1244, 10)


In [193]:
correct_predictions['english_text'] = correct_predictions.apply(translate_to_english, axis=1)
correct_predictions.to_csv('../results/correct_predictions_v2_to_analyse.csv')

--------------------- AfD-Sprecher Martin Renner kandiert auf der NRW-Liste für den Bundestag auf Platz 1. Pretzell setzt auf Export aus Hamburg Kay Gottschalk.
--------------------- Gaaaanz knapp bei der Stichwahl für Platz 1 der Liste der #AfD #NRW für den #Bundestag: Martin Renner siegt soeben vor Kay Gottschalk.
--------------------- Gratulation meinem Hamburger Parteikollegen Kay Gottschalk zu Listenplatz 4 der AfD NRW zur Bundestagswahl. #afd  #btw2017 #afdessen
---------------------   Die Grünen sind Linksextremisten.
---------------------  Wer grün wählt, unterstützt den Linksextremismus, nicht die Natur, siehe EEG, Windräder, Klimalüge.
---------------------   Die Direktübertragung sollte künftig organisiert werden...
---------------------   Schon mal drüber nachgedacht WER das bezahlt? - DU zahlst das (solltest Du Steuern zahlen) und ich, etc. DAS findest Du in Ordnung ?
--------------------- Wieder eine tolle Wahlkampfveranstaltung der  mit  und  bei der  #TrauDich #AfD 
---

---------------------    Meines Wissens nach ist kostenlose Werbung eine geldwerte Zuwendung & damit eine Parteispende, aber das klären Gerichte #AfDSwissConnection
--------------------- NEW VIDEO by Jörg Meuthen Wir sind im Sorge um unsere Heimat, Vaterland und Zukunft unserer...
---------------------    Sie ist ein großer Mensch
---------------------   Verzeihen Sie den Fehler, ich meine selbstverständlich Herrn #Arppe
---------------------    Ein EU Einwanderungsgesetz, ja, unbedingt. Hat aber mit Asylrecht kaum etwas zu tun. Und Asylbewerber einfach zurückweisen ist unmenschlich.
---------------------  Das eine: Zum anderen würde ich mal empfehlen zu go-ogeln nach: dr alice weidel goldman sachs; Finanzkartell NO WAY!
--------------------- NEW VIDEO by Wahlkampfveranstaltung der #AfD Südthüringen mit Jörg Meuthen, Stephan Brandner
---------------------    Und was soll jetzt genau damit sein? Richtig "Rechts" sowas oder?
---------------------   Falls Sie Ihren 'illegal', 'OECD' Quats

---------------------  Wech mit dem Halunken
---------------------  Wer Pol Pot zur Machtübernahme gratuliert, der steht mit Heinrich Himmler moralisch auf einem Podest.
---------------------   Linksversifft und dumm ... und von Demokratie keinen blassen Schimmer ...
---------------------   Natürlich. In den Augen der #afd ist die NSU Patrioten und die Opfer nur morgenländische Subjekte. #noafd 
---------------------   Sie beten aber wohl nicht den christlichen Gott an bei so unchristlichen Kommentaren oder? Bitten Sie Gott lieber um Benehmen und Respekt!
---------------------   Sie zitieren wirklich einen Blog der die Menschenjagt durch Linksradikale fördert?
--------------------- Jörg Meuthen zerfetzt die Bundesregierung in der Luft - Abensberg 04.09.17  via 
--------------------- NEW VIDEO by AFD-Television:#AfD   Prof  Dr  Jörg Meuthen   Audio Kolumne vom 30 09 2017 Eine historische Woche
---------------------    Die braucht niemand.Sie meinen also die "fremden Völker", die nach D.

---------------------  Komisch im Duden(Onlinefassung) wird national als Synonym für nationalistisch verwendet. D.h. dann wohl aus #AfD-Sicht.. #Lügenduden 🤣
--------------------- Als SLDP Politikerin rufe Ich nachWiderstand&Resistance gegen die AfD ! AfD ist Staatsfeind nr1in BRD! Gemeinsam gegen alle AfD Staatsfeinde
--------------------- LKW tötet. Messer tötet. Waffe tötet. Nein, der MENSCH der diese bedient, tötet. #Schrottpresse #noIslam #AfD 
--------------------- #NSAfD Anspruch und Wirklichkeit ... - #NSAfD Anspruch und Wirklichkeit ... AfD-Chef Alexander Gauland hatte Ay... 
---------------------   ja, und da wissen sie leider auch ganz genau, sie sind nämlich nicht so doof wie alexander gauland aussieht.
--------------------- Dr. Alexander #Gauland, Fraktionsvorsitzender der #AfD im #Bundestag, lehnt die Forderungen der Präsidentin des Umweltbundesamtes, Maria Krautzberger (SPD), unter anderem nach einer allgemeinen Maut für alle PKW auf sämtlichen Straßen, entschieden ab.➡️

---------------------  Deutsche haben Millionen Juden getötet, weil sie glaubten, das Judentum als DAS Problem schlechthin identifiziert zu haben. Heute gibt es Deutsche, die glauben, den Islam als DAS Problem schlechthin identifiziert zu haben. Die Hetze der #AfD ist Goebbels' Arschgeburt.
---------------------  doch:wenn der bürger diese aufgabe übernimmt..so wie früher..
---------------------  Ein offen zur Schau gestelltes Christentum auch, Kreuzzug mit umgekehrtem Vorzeichen
---------------------  Es gibt 80 Millionen Deutsche, davon ca. 1 Million Gutmenschen. Diese eine Million verteten die Deutschen Medien. Ein Präsident von Amerika wird auf das übelste von der Politik und den Medien verteufelt. Frau Merkel wird von den selben vergöttert. Dieses Spiel verstehe ich nicht.
---------------------  Kein Experte in Politik zitiert keinen Experten in Politik! #Lagerfeld
---------------------  Toll, wie sich Herr Professor jetzt für jüdische Mitbürger einsetzt! Schaffen Sie es auch, Ihr

...

--------------------- Die vielen Fälle, von denen wir nie erfahren, weil nicht angezeigt/ eingestellt/Opfer mit Überleben beschäftigt... mir ist grad schlecht
--------------------- einfach lesen... 
--------------------- Ich beschreibe jetzt eine Szene, für die ich seinerzeit eigens eine Besichtigung im Flughafen Tempelhof mitgemacht habe im März 2016. #Schreibnacht
--------------------- Ich habe keinen Bock mehr, keinem gut genug zu sein.
--------------------- Trolle, die für das Trollen bezahlt werden, nennt man Astroturfer.#infotweet
---------------------    Meiner ist nur halb besetzt, aber wenn ich mehr als 30 fahre, beschwert sich mein Kleiner.
---------------------   Ein paar wenige haben S-Pedelecs mit Nummernschild, sind aber eher die Ausnahme.
---------------------  Aber die AfD will doch nur das wiederbringen, was sich im Nachhinein als schlecht herausgestellt hat. Schlecht gewählt, würde ich behaupten.
---------------------  Dann schau dir 1fach den Rest der Serie an und er

--------------------- #Abu_DhabiAuch bei der Zusammenarbeit in Kunstprojektensollte man die Fragen von #Demokratie und #Freiheit nicht aus dem Blick verlieren.#FreeDenis#FreeMesale#FreeThemAll, (auch schweizer #Journalisten in #Wüstenstaaten!)
--------------------- Eine Anschauempfehlung für alle, die die letzten Wochen die Auseinandersetzungen in der Linken irgendwie verfolgt haben. Ich finde, damit kann es wieder vorwärts gehen. 
--------------------- Ich würde mich gerne beruflich neu orientieren und mein (mehr als nur) Hobby Politik zu meinem Lebensinhalt machen. Falls jemand in der Partei DIE LINKE oder bei der RLS Interesse an einem Mitarbeiter hat, kann sich gerne bei mir melden. 
---------------------   Oh, den Grünen hab ich wohl übersehen. (ruhig konkreter antworten, was du meinst) wär auch zu schön gewesen. Ändert nichts an meiner Grundaussage.
---------------------  Den Fachkräftemangel habe ich als frischer Akademiker am eigenen Leib zu spüren bekommen. 7 Monate Jobsuche.


---------------------   Laschet wünscht sich mehr Sachlichkeit… WTF! Ist das nicht der Mann, der gesagt hat: "Ich schaffe Euer Sozialticket ab. Bätschi! Bätschi! Bätsch💥" … 👿👿👿 🤕 "Oke. If faffe Euer Fosialticket doch nif abb…" 😔
--------------------- Welches Projekt aus ihrem Wahlprogramm sollte die SPD in einer möglichen neuen Großen Koalition auf jeden Fall umsetzen?  #CiveyFrage via  
---------------------    Es braucht: Gewissenhafte Diagnostik und bei ADHS dann multimodale Therapie, bei der Ritalin oft ein wichtiger Baustein ist.
--------------------- Die USA will wieder eine bemannte #Mondmission. Sie sollten den gesamten Trump-Clan mitnehmen, am besten auch Kim Jong-un, Erdogan so wie die Reste des Kapitalismus  - und dann dort vergessen!  
--------------------- In zwei Tagen kann ich 17.000 Euro für den iMac Pro lassen, wenn ich möchte… :D
--------------------- Wie ich Magneten aus einer Hülle ausbaute. 
--------------------- "Die sich vor der Sonne fürchten müssen"Update zu „L

---------------------  Geht schon seit Jahren so, und die Politik greift nicht durch. Man hat das Gefühl, die linke Szene genießt Narrenfreiheit. 😡
---------------------   Die Linke hat sich schon eine ganze Weile unwählbar gemacht wie alle anderen Altparteien auch!Aber das jetzt ist doch ein Geschmäckle!😖👎👎
---------------------    Als ich heute morgen auf der A66,Richtung ffm gefahren. Wurde ich von Rollsplitt beschossen. Die hatten die linke Spur geflutet 😡
---------------------  Die Linke ... danach hab ich nicht mehr weitergelesen 👍
---------------------  Wir sprechen immer über rechte Gewalt, die linke Gewalt darf auch nicht geduldet werden! Dank an jeden einzelnen Polizisten 👍
---------------------  Die Linke kann natürlich direkt raus aus dem Bundestag. Dann haben die gewalttätigen Idioten auch kein Sprachrohr mehr. ❤️
---------------------   Stellungnahme von Die #Linke zu Hamburg. #G20 👎 
--------------------- 100%👍 „Es ist zum Kotzen, dass die Linke nicht bereit &fähig ist, 

--------------------- Gregor Gysi ist ein guter Mensch #lastRT
--------------------- Da haben wir es: Gregor Gysi ist ein Nazi... 😆N24Verified accountLinken-Abgeordnete erklärt SED zur rechten Partei 
--------------------- Herr Gregor Gysi ist ein sehr guter Redner und Politiker. Doch DIE LINKE besteht nicht nur aus Herrn Gysi. One Man Show is out.  Ich seh und lese viel  dieser Tage. Es macht immer wieder Laune. Eine positive Kraft! Bin froh, dass alle an einem Strang ziehen.  …


## Analysis of words based on the PMI score

In [210]:
from collections import Counter
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from matplotlib import font_manager
from sklearn.feature_extraction.text import CountVectorizer
from collections import OrderedDict

In [217]:
def get_top_n_features_based_pmi(n, left_tweets, right_tweets, left=True):
    n_left = len(left_tweets)
    n_right = len(right_tweets)
    left_document = ' '.join(left_tweets)
    right_document = ' '.join(right_tweets)
    document = ' '.join([left_document,right_document])
    cv = CountVectorizer(stop_words='english')
    cv_fit = cv.fit_transform([left_document, right_document])
    features = cv.get_feature_names()
    
    counts = cv_fit.toarray()
    print(len(features), counts.shape)
    pmi = []
    for index, f in enumerate(features):
        N = (n_left + n_right)
        p_x = (counts[0][index]+counts[1][index])/N
        p_x_left = counts[0][index]/N
        p_x_right = counts[1][index]/N
        p_left = n_left/N
        p_right = n_right/N
        
        try:
            pmi_x_left = math.log2(p_x_left/(p_x * p_left))
        except ZeroDivisionError:
            pmi_x_left = 0
        except Exception as e:
            pmi_x_left = 0
        try:
            pmi_x_right = math.log2(p_x_right/(p_x * p_right))
        except ZeroDivisionError:
            pmi_x_right = 0
        except Exception as e:
            pmi_x_right = 0
        
        combined_score = (pmi_x_left - pmi_x_right)
        if left:
            combined_score = (pmi_x_left - pmi_x_right) * p_x_left
        else:
            combined_score = (pmi_x_right - pmi_x_left) * p_x_right
            
        pmi.append({'feature':f,
                    'pmi_hate': pmi_x_left,
                    'pmi_neutral': pmi_x_right,
                    'score': combined_score})
    
    pmi_df = pd.DataFrame(pmi)
    pmi_df = pmi_df.sort_values(by=['score'], ascending=False)
    print('left: ', left, pmi_df.tail(5))
    top_n = pmi_df.head(n)
    print('top_n: ', top_n)
    return top_n

In [218]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print(wrong_predictions.shape)
wrong_predictions = wrong_predictions.dropna()
print(wrong_predictions.shape)
top_n_left = get_top_n_features_based_pmi(100,
                             wrong_predictions[wrong_predictions['labels']==0]['english_text'].tolist(),
                             wrong_predictions[wrong_predictions['labels']==1]['english_text'].tolist(),
                            left=True)
top_n_right = get_top_n_features_based_pmi(100,
                             wrong_predictions[wrong_predictions['labels']==0]['english_text'].tolist(),
                             wrong_predictions[wrong_predictions['labels']==1]['english_text'].tolist(),
                            left=False)

(685, 11)
(685, 11)
2545 (2, 2545)
left:  True      feature  pmi_hate  pmi_neutral     score
2280   think -0.558274     0.429985 -0.008656
1029   great -0.640736     0.473054 -0.009756
1671  people -0.640736     0.473054 -0.011382
1333    left -0.892275     0.584085 -0.015087
1357    like -0.558274     0.429985 -0.017313
top_n:              feature  pmi_hate  pmi_neutral     score
985         germany  0.664119    -1.444484  0.043096
655             did  0.596303    -1.164376  0.028274
1977           said  0.721834    -1.749339  0.021645
84         actually  0.721834    -1.749339  0.021645
1266           just  0.285264    -0.389443  0.018714
546         country  0.581657    -1.111909  0.017307
1379           long  0.681192    -1.526946  0.016118
1239        jamaica  0.681192    -1.526946  0.016118
2279         things  0.681192    -1.526946  0.016118
950            free  0.681192    -1.526946  0.016118
1593             oh  0.944227     0.000000  0.012406
1096           help  0.429654    

/nethome/aanegundi/miniconda3/envs/env3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/nethome/aanegundi/miniconda3/envs/env3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [220]:
top_n_left.to_csv('../results/top_n_left_wrong_predictions.csv')
top_n_right.to_csv('../results/top_n_right_wrong_predictions.csv')

In [221]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print(correct_predictions.shape)
correct_predictions = correct_predictions.dropna()
print(correct_predictions.shape)
top_n_left = get_top_n_features_based_pmi(100,
                             correct_predictions[correct_predictions['labels']==0]['english_text'].tolist(),
                             correct_predictions[correct_predictions['labels']==1]['english_text'].tolist(),
                            left=True)
top_n_right = get_top_n_features_based_pmi(100,
                             correct_predictions[correct_predictions['labels']==0]['english_text'].tolist(),
                             correct_predictions[correct_predictions['labels']==1]['english_text'].tolist(),
                            left=False)

(3570, 11)
(3566, 11)
7982 (2, 7982)
left:  True         feature  pmi_hate  pmi_neutral     score
3076     german -1.155279     0.373182 -0.005572
1140  bundestag -0.802608     0.297011 -0.005859
3078    germany -1.205045     0.382238 -0.010238
4620     merkel -0.746960     0.282757 -0.010684
309         afd -2.354563     0.517016 -0.045095
top_n:               feature  pmi_hate  pmi_neutral     score
7679     wagenknecht  1.505774    -6.122528  0.226753
4226            left  1.143596    -1.505868  0.207291
4318           linke  1.492848    -5.162421  0.201562
2069             die  1.398305    -3.016650  0.099046
6645          social  1.218151    -1.789146  0.047226
3262            gysi  1.519320     0.000000  0.045162
3204          gregor  1.519320     0.000000  0.043032
6272           sahra  1.519320     0.000000  0.042606
877            bernd  1.322283    -2.350688  0.042230
6729             spd  0.838780    -0.791994  0.033384
4058         kipping  1.519320     0.000000  0.028120
1

/nethome/aanegundi/miniconda3/envs/env3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/nethome/aanegundi/miniconda3/envs/env3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [222]:
top_n_left.to_csv('../results/top_n_left_correct_predictions.csv')
top_n_right.to_csv('../results/top_n_right_correct_predictions.csv')